# HW2 - – Machine Learning in Healthcare 336546

*Bar Goldner & Daria Hasin*

# Theory Questions

1. Accuracy is calculated using this formula: [(TP + TN) / (TP + TN + FP + FN)]. We can see that the accuracy is based on the performance statistics. But as we saw in class, if we have a population of 900 patients of which 100 are AF and 800 are not AF, the accuracy that was calculated is 0.95 which is high, but the data is not balanced as we saw in the confusion matrix, then learning will be poor. Therefore, we prefer to calculate the **performance** statistics and treat them separately rather than relying on model accuracy.

2. *Definitions*:

    **Classifier-1**: uses only Blood Pressure (BP) and Ejection Fraction (EF) features.
    
    **Classifier-2**: uses all of the features (Age, serum sodium, serum creatinine, gender, smoking, BP, EF, anemia, platelets, Creatinine Phosphokinase (CPK) and diabetes).
    
    *Comparison between Classifier-1 and Classifier-2*: 
    #### Classifier-1
    ##### Pros: 
    * Low complexity - less features, less complex model.
    * Less patients are needed to train the model.

    ##### Cons:
    * We dismiss potentially relevant features that we can use for the classification and it can be less accurate.
    * Can lead to underfitting and high bias.

    #### Classifier-2
    ##### Pros: 
    * We use a lot of features so we don't loose any data about the patients and may have better predictions.

    ##### Cons:
    * We use all of the features and there is a great probability that not all of them are relevant.
    * High complexity and time consuming.
    * Can lead to overfitting and high variance.

3. Two differences between what we will obtain from linear SVM and LR:

    1 - The chance of overfitting in the SVM model is lower than in the linear regression model because we have supporting vectors in SVM, that gives the separating line "safe space" (margin) from the samples, so it will not be closer to specific class - unlike LR.

    2 - The time that takes to run linear SVM is much higher than linear regression, while the results will probably be the same - as Moran mentioned in Tutorial 6. That means that we can obtain our results faster with LR without jeopardizing the quality of the results.

4. First, we will define Logistic Regression (LR) and Support Vector Machine (SVM):

    **Logistic Regression** is a statistical model that separtes the data to several classes. LR is calculating the probabilities of the patient (for example) to be assighned to a specific class and choosing the class with the *higher probability*. 
    Note: this is a *linear* model.

    **Support Vector Machine** is a model that separtes the data to several classes (like LR),  but SVM takes the *higher distance* between two closest data samples and findes the best hyperplane that separtes the data. Also, in this model there is a use of support vectors that defines the margin to the hyperplane.
    Note: this model can be *linear and non-linear*, it dependes on the kernal function (linear / polynominal / rbf) and the dimantion that we are choosing for the calculations. 
    For example, for kernal=polynominal and d=2 we get a line that separtes the data (linear SVM). 

    Therefore, linear SVM is based on geometrical properties (distance) of the data while LR relies on statistical preoperties (probabilities).
    This difference affect their hyper-parameters tuning, especially on choosing the parameter C. *In SVM, C affects on accuracy*, because it is focused on the trade-off between maximizing the distance between the hyperplane and the support vectors (the margin) and minimizing the misclassified samples.
    *In LR, C affects on complexity*, because it is focused on the trade-off between maximizing the number of features in use and simplifying the model.

# Coding Assignment

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold as SKFold
import scipy.stats as stats
from sklearn.metrics import plot_confusion_matrix, roc_auc_score, confusion_matrix, plot_roc_curve
from tqdm import tqdm
import sys
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

Data downloading

In [ ]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv

In [ ]:
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')

In [ ]:
df

## Preprocessing of the data

First we will check if there are nan or non-numeric values in the dataset & replace with random value from the same feature:

In [ ]:
df = df.replace(regex='[^0-9]', value=np.nan)

for feature in df.columns:
    one_feature = df.loc[:, feature].tolist()

    for idx, x in enumerate(one_feature):
        while np.isnan(x):
            rand_idx = np.random.choice(len(one_feature))
            x = one_feature[rand_idx]
            df.loc[idx, feature] = x
df

Dropping the feature 'time' because follow-up time is not a relavent feature for heart failure preditions:

In [ ]:
df_no_time = df.copy()
df_no_time.drop(columns='time', inplace=True)

Now we will plot an histogram to check outliers:

In [ ]:
ax = df_no_time.hist(figsize=(15,15), color='purple')
x_units = ['Years', 'N.U.', 'mcg/L', 'N.U.', '%', 'N.U.', 'platelets/mL', 'mg/dL', 'mEq/L', 'N.U.', 'N.U.', 'days', 'N.U.']
count = 0

for i in range(ax.shape[0]):
    for j in range(ax.shape[1]):
        ax[i][j].set_ylabel('Counts')
        try:
            ax[i][j].set_xlabel(f'{x_units[count]}')
        except:
            pass
        count += 1

No outliers were found.

Lastly, we will plot an heatmap to check correlation between features:

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(df_no_time.corr(), annot = True)

We can see that the highest correlation of the death event feature (by descending order) is with: serum creatinine, ejection fraction, age and serum sodium.

### Test-train split

In [ ]:
X = df_no_time.copy()
y = X['DEATH_EVENT'].values.ravel()
X.drop(columns='DEATH_EVENT', inplace=True) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

## Visualization and exploration of the data
To show that the distribution of the features is similar between test and train, we chose to plot the distribution of every feature in the two sets:

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(10,20))
ax = axs.ravel()
features = df.columns
x_features = X.columns

for idx in range(len(ax)):
    ax[idx].hist(X_train.iloc[:, idx], density=True, alpha=0.5, label='train', color='black')
    ax[idx].hist(X_test.iloc[:, idx], density=True, alpha=0.5, label='test', color='purple')
    ax[idx].legend(loc='upper right')
    ax[idx].set_title(features[idx])
plt.show()

Also, we chose to calculate the mean of each feature and compare between the test set and train set:

In [ ]:
d_table = pd.DataFrame(index=x_features, columns=['Train', 'Test', 'Delta'])

for feat in x_features:
    feat_mean_train = np.round(X_train.loc[:, feat].mean(), 2)
    d_table.loc[feat, 'Train'] = feat_mean_train
    
    feat_mean_test = np.round(X_test.loc[:, feat].mean(), 2)
    d_table.loc[feat, 'Test'] = feat_mean_test
    
    d_table.loc[feat, 'Delta'] = abs(feat_mean_train-feat_mean_test)

feat = 'DEATH_EVENT'
feat_mean_y_train = np.round(np.mean(y_train), 2)
d_table.loc[feat, 'Train'] = feat_mean_y_train

feat_mean_y_test = np.round(np.mean(y_test), 2)
d_table.loc[feat, 'Test'] = feat_mean_y_test

d_table.loc[feat, 'Delta'] = abs(feat_mean_y_train-feat_mean_y_test)

d_table

#### What issues could imbalance the features between train and test cause? 
If the train's features distribution is different than the test's features distribution, the model can calculate the prediction in a wrong way. The model calculation is based on the learning from the train data set, and it applies the learning on the test data set, so if there are different distribution, the applying part will be wrong.

#### How could you solve the issue?
We can use the "stratify" attribute that in the " train_test_split" function which make the data split in a stratified fashion, using the class labels.

### Plots that show the relationship between feature and label

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15,10))
ax = axs.ravel()
boolian_features = ['anaemia', 'high_blood_pressure', 'diabetes', 'sex' ,'smoking']
check_feat = 'DEATH_EVENT'

for idx, feat in enumerate(boolian_features):
    countplot = sns.countplot(ax=ax[idx] ,x=feat, data=df[[feat, check_feat]], hue=check_feat, palette='Set2')
    if feat == 'sex':
        countplot.set_xticklabels(['Women', 'Men'])
    else:
        countplot.set_xticklabels(['No', 'Yes'])
    
    countplot.legend(labels=['No', 'Yes'], title='Death event')
    

###  Additional plots to see correlation between binary and non-binary features

In [ ]:
sns.scatterplot(data=df, x="age", y="high_blood_pressure", hue="DEATH_EVENT")

In [ ]:
sns.scatterplot(data=df, x="creatinine_phosphokinase", y="diabetes", hue="DEATH_EVENT")

In [ ]:
sns.scatterplot(data=df, x="ejection_fraction", y="high_blood_pressure", hue="DEATH_EVENT")

#### Was there anything unexpected?
As we can see on the plots that show the relationship between feature and label, most of the binary feature divided is similar in terms of number of deaths. It was unexpected to find out that there is not linear correlation of death between the two features "age" and "high_blood_pressure" and also between the two features "age" and "diabetes".

#### Are there any features that you feel will be particularly important to your model? Explain why.
When we went through the dataset, we noticed that the smaller the value of the feature "time", the higher the deaths. When we thought about the meaning of the feature, we realized that this feature represents the patients' follow-up time, so it makes sense. After much thought we decided to remove this feature from the dataset even though we know it will hurt the chances of predicting correctly. That's because this feature does not really represent a meaning to any person who has not taken part of the data.

## Machine Learning Models
### Linear model - Logistic Regression

In [ ]:
def check_penalty(penalty='none'):
    if penalty == 'l1':
        solver='liblinear'
    if penalty == 'l2' or penalty == 'none':
        solver='lbfgs'
    return solver

In [ ]:
scaler = MinMaxScaler()
C = np.array([0.001, 0.01, 1, 10, 100, 1000])
K = 5 
penalty = ['l1', 'l2']
dict_values = list()

kf = SKFold(n_splits=K, random_state=10, shuffle=True)

with tqdm(total=len(C), file=sys.stdout) as pbar:
    for c in C:
        pbar.set_description('processed: %d/%d' % ((1 + idx), len(C)))
        pbar.update(1)
        for p in penalty:
            solver = check_penalty(p)
            logreg = LogisticRegression(solver=solver, penalty=p, C=c, max_iter=10000, multi_class='ovr')
            loss_val_vec = np.zeros(K)
            k = 0
            for train_idx, val_idx in kf.split(X_train, y_train):
                x_train_tmp, x_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_train_tmp, y_val = y_train[train_idx], y_train[val_idx]

                x_train_tmp = scaler.fit_transform(x_train_tmp)
                x_val = scaler.transform(x_val)

                logreg.fit(x_train_tmp, y_train_tmp)
                y_pred_val = logreg.predict_proba(x_val)
                loss_val_vec[k] = log_loss(y_val, y_pred_val)
                k += 1
            dict_values.append({'C': c, 'penalty': p, 'mu': np.mean(loss_val_vec), 'sigma': np.std(loss_val_vec)})

loss_arr = [x['mu'] for x in dict_values]
argMinLoss = np.argmin(loss_arr)
best_c = dict_values[argMinLoss]['C']
best_penalty = dict_values[argMinLoss]['penalty']
best_loss = dict_values[argMinLoss]['mu']
print(f'The chosen C: {best_c} and the chosen penalty: {best_penalty}')

In [ ]:
for d in dict_values:
    x = np.linspace(0, d['mu'] + 3 * d['sigma'], 1000)
    plt.plot(x, stats.norm.pdf(x, d['mu'], d['sigma']), label="p = " + d['penalty'] + ", C = " + str(d['C'])) 
    plt.title('Gaussian distribution of the loss')
    plt.xlabel('Average loss')
    plt.ylabel('Probabilty density')
    plt.ylim(0, 70)

plt.legend()
plt.show()

In [ ]:
solver = check_penalty(best_penalty)
logreg = LogisticRegression(solver=solver, penalty=best_penalty, C=best_c, max_iter=10000, multi_class='ovr')
logreg.fit(X_train, y_train)

plot_confusion_matrix(logreg, X_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

In [ ]:
calc_TN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 0]
calc_FP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[0, 1]
calc_FN = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 0]
calc_TP = lambda y_true, y_pred: confusion_matrix(y_true, y_pred)[1, 1]

In [ ]:
def check_stats(y_test, y_pred_test):
    TN = calc_TN(y_test, y_pred_test)
    FP = calc_FP(y_test, y_pred_test)
    FN = calc_FN(y_test, y_pred_test)
    TP = calc_TP(y_test, y_pred_test)
    Se = TP/(TP+FN)
    Sp = TN/(TN+FP)
    PPV = TP/(TP+FP)
    NPV = TN/(TN+FN)
    Acc = (TP+TN)/(TP+TN+FP+FN)
    F1 = (2*Se*PPV)/(Se+PPV)
    return TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1

In [ ]:
y_pred_test = logreg.predict(X_test)
y_pred_proba_test = logreg.predict_proba(X_test)


TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.4f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_test)))
print('Loss is: {:.4f}.'.format(best_loss))

In [ ]:
plot_roc_curve(logreg, X_test, y_test)
plt.grid(False)

#### Non-linear model - Support Vector Machine

In [ ]:
svc = SVC(probability=True, random_state=10, max_iter=1000)
pipe = Pipeline(steps=[('scale', MinMaxScaler()), ('svm', svc)])

svm_nonlin = GridSearchCV(estimator=pipe,
             param_grid={'svm__kernel':['rbf', 'poly'], 'svm__C':C, 'svm__degree':[2, 3, 4]},
             scoring='roc_auc', 
             cv=kf, refit='f1', verbose=3, return_train_score=True)
svm_nonlin.fit(X_train, y_train)
best_svm_nonlin = svm_nonlin.best_estimator_
print()
print('Best parameters for SVM:', svm_nonlin.best_params_)

y_pred_test = best_svm_nonlin.predict(X_test)
y_pred_proba_test = best_svm_nonlin.predict_proba(X_test)
plot_confusion_matrix(best_svm_nonlin, X_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print()
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.2f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_proba_test[:,1])))

In [ ]:
plot_roc_curve(svm_nonlin, X_test, y_test)
plt.grid(False)

#### What performs best on this dataset? Linear or non-linear models?
**LR:**

*Accuracy:* 0.8000. 

*F1:* 0.6471.

*AUROC:* 0.7407.

**SVM:**

*Accuracy:* 0.6667.

*F1:* 0.4444. 

*AUROC:* 0.7189.

We can see that the *linear model* performed better than the non-linear model.

### Feature Selection

In [ ]:
rfc = RandomForestClassifier(random_state=10)
rfc.fit(X_train, y_train)

y_pred_test = rfc.predict(X_test) #NOTICE NOT TO USE THE STANDARDIZED DATA.
y_pred_proba_test = rfc.predict_proba(X_test)
plot_confusion_matrix(rfc,np.array(X_test),y_test, cmap=plt.cm.Blues)
plt.grid(False)
TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.2f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_proba_test[:,1])))

roc_score = []
plt.figure()
ax = plt.gca()
plot_roc_curve(rfc, X_test, y_test, ax=ax)

We can see that the accuracy and the f1 in the Random Forast model is lower than the accuracy of LR model, but AUROC is higher.

In [ ]:
importances = rfc.feature_importances_
forest_importances = pd.Series(importances, index=x_features)
std = np.std([tree.feature_importances_ for tree in rfc.estimators_], axis=0)

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

#### What are the 2 most important features according to the random forest.
The 2 most important features are serum creatinine and ejection fraction.

#### Does this match up exactly with the feature exploration you did?
It is match up  with the feature exploration becuse when we checked the correlation both of those features showed the highest values in absolute value.

## Data Separability Visualization

In [ ]:
n_components = 2
pca = PCA(n_components=n_components, whiten=True)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Z_train = pca.fit_transform(X_train_scaled)
Z_test = pca.transform(X_test_scaled)

In [ ]:
def plt_2d_pca(X_pca,y):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, aspect='equal')
    ax.scatter(X_pca[y==0, 0], X_pca[y==0, 1], color='b')
    ax.scatter(X_pca[y==1, 0], X_pca[y==1, 1], color='r')
    ax.legend(('Alive','Dead'))
    ax.plot([0], [0], "ko")
    ax.arrow(0, 0, 0, 1, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.arrow(0, 0, 1, 0, head_width=0.05, length_includes_head=True, head_length=0.1, fc='k', ec='k')
    ax.set_xlabel('$U_1$')
    ax.set_ylabel('$U_2$')
    ax.set_title('2D PCA')

In [ ]:
plt_2d_pca(Z_test,y_test)

#### How separable is your data when reduced to just two features?
The data is not separable because the 'Dead' and 'Alive' classes are mixed together and do not create typical clusters, as an opposite to our expectations.

In [ ]:
dict_values = list()

with tqdm(total=len(C), file=sys.stdout) as pbar:
    for c in C:
        pbar.set_description('processed: %d/%d' % ((1 + idx), len(C)))
        pbar.update(1)
        for p in penalty:
            solver = check_penalty(p)
            logreg = LogisticRegression(solver=solver, penalty=p, C=c, max_iter=10000, multi_class='ovr')
            loss_val_vec = np.zeros(K)
            k = 0
            for train_idx, val_idx in kf.split(Z_train, y_train):
                z_train_tmp, z_val = Z_train[train_idx], Z_train[val_idx]
                y_train_tmp, y_val = y_train[train_idx], y_train[val_idx]

                z_train_tmp = scaler.fit_transform(z_train_tmp)
                z_val = scaler.transform(z_val)

                logreg.fit(z_train_tmp, y_train_tmp)
                y_pred_val = logreg.predict_proba(z_val)
                loss_val_vec[k] = log_loss(y_val, y_pred_val)
                k += 1
            dict_values.append({'C': c, 'penalty': p, 'mu': np.mean(loss_val_vec), 'sigma': np.std(loss_val_vec)})

loss_arr = [x['mu'] for x in dict_values]
argMinLoss = np.argmin(loss_arr)
best_c = dict_values[argMinLoss]['C']
best_penalty = dict_values[argMinLoss]['penalty']
best_loss = dict_values[argMinLoss]['mu']
print()
print(f'The chosen C: {best_c} and the chosen penalty: {best_penalty}')
solver = check_penalty(best_penalty)
logreg = LogisticRegression(solver=solver, penalty=best_penalty, C=best_c, max_iter=10000, multi_class='ovr')
logreg.fit(Z_train, y_train)

plot_confusion_matrix(logreg, Z_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.2f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_proba_test[:,1])))
print('Loss is: {:.4f}.'.format(best_loss))

In [ ]:
svm_nonlin.fit(Z_train, y_train)
best_svm_nonlin = svm_nonlin.best_estimator_
print()
print('Best parameters for SVM:', svm_nonlin.best_params_)

y_pred_test = best_svm_nonlin.predict(Z_test)
y_pred_proba_test = best_svm_nonlin.predict_proba(Z_test)
plot_confusion_matrix(best_svm_nonlin, Z_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print()
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.2f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_proba_test[:,1])))

In [ ]:
dict_values = list()

two_feat_X_train = X_train.loc[:,{'serum_creatinine','ejection_fraction'}]
two_feat_X_test = X_test.loc[:,{'serum_creatinine','ejection_fraction'}]

kf = SKFold(n_splits=K, random_state=10, shuffle=True)

with tqdm(total=len(C), file=sys.stdout) as pbar:
    for c in C:
        pbar.set_description('processed: %d/%d' % ((1 + idx), len(C)))
        pbar.update(1)
        for p in penalty:
            solver = check_penalty(p)
            logreg = LogisticRegression(solver=solver, penalty=p, C=c, max_iter=10000, multi_class='ovr')
            loss_val_vec = np.zeros(K)
            k = 0
            for train_idx, val_idx in kf.split(two_feat_X_train, y_train):
                x_train_tmp, x_val = two_feat_X_train.iloc[train_idx], two_feat_X_train.iloc[val_idx]
                y_train_tmp, y_val = y_train[train_idx], y_train[val_idx]
                x_train_tmp = scaler.fit_transform(x_train_tmp)
                x_val = scaler.transform(x_val)

                logreg.fit(x_train_tmp, y_train_tmp)
                y_pred_val = logreg.predict_proba(x_val)
                loss_val_vec[k] = log_loss(y_val, y_pred_val)
                k += 1
            dict_values.append({'C': c, 'penalty': p, 'mu': np.mean(loss_val_vec), 'sigma': np.std(loss_val_vec)})

loss_arr = [x['mu'] for x in dict_values]
argMinLoss = np.argmin(loss_arr)
best_c = dict_values[argMinLoss]['C']
best_penalty = dict_values[argMinLoss]['penalty']
best_loss = dict_values[argMinLoss]['mu']
print()
print(f'The chosen C: {best_c} and the chosen penalty: {best_penalty}')

solver = check_penalty(best_penalty)
logreg = LogisticRegression(solver=solver, penalty=best_penalty, C=best_c, max_iter=10000, multi_class='ovr')
logreg.fit(two_feat_X_train, y_train)

plot_confusion_matrix(logreg, two_feat_X_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

y_pred_test = logreg.predict(two_feat_X_test)
y_pred_proba_test = logreg.predict_proba(two_feat_X_test)


TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.4f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_test)))
print('Loss is: {:.4f}.'.format(best_loss))

In [ ]:
svm_nonlin.fit(two_feat_X_train, y_train)
best_svm_nonlin = svm_nonlin.best_estimator_
print()
print('Best parameters for SVM:', svm_nonlin.best_params_)

y_pred_test = best_svm_nonlin.predict(two_feat_X_test)
y_pred_proba_test = best_svm_nonlin.predict_proba(two_feat_X_test)
plot_confusion_matrix(best_svm_nonlin, two_feat_X_test, y_test, cmap=plt.cm.Blues)
plt.grid(False)

TN, FP, FN, TP, Se, Sp, PPV, NPV, Acc, F1 = check_stats(y_test, y_pred_test)
print('Sensitivity is {:.4f}. \nSpecificity is {:.4f}. \nPPV is {:.2f}. \nNPV is {:.4f}. \nAccuracy is {:.4f}. \nF1 is {:.4f}. '.format(Se,Sp,PPV,NPV,Acc,F1))
mcc = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
print('MCC is {:.4f}.'.format(mcc))
print('AUROC is {:.4f}.'.format(roc_auc_score(y_test, y_pred_proba_test[:,1])))

#### What performs better? 2 features or the reduced dimensionality.

#### reduced dimensionality performance statistics:

**LR:**

*Accuracy:* 0.7833. 

*F1:* 0.6061.

*AUROC:* 0.8299.

**SVM:**

*Accuracy:* 0.7167.

*F1:* 0.1905. 

*AUROC:* 0.7150.


#### 2 feature performance statistics:

**LR:**

*Accuracy:* 0.7333. 

*F1:* 0.4667.

*AUROC:* 0.6354.

**SVM:**

*Accuracy:* 0.7333. 

*F1:* 0.5294. 

*AUROC:* 0.8151.


We can see that the *dimensionality reduced LR model* preformed best. Also, we see that the 2 features SVM preformed better than dimensionality reduced SVM model.


### That's all :)